In [58]:
### Here we would use cyclic peptides from this paper: Crystal Structures of Protein-Bound Cyclic Peptides
### Here we develop several clustering strategies using both structural and physico-chemical properties of the peptides
### Finally the clustering models with be evaluated by performing a structure BLAST agains the PDB database
### to determine how well our model would generalize

In [59]:
from platform import python_version
print(python_version())

%pip install --upgrade --user notebook

3.10.6
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [60]:
%pip install --user scikit-learn peptides biopython numpy pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [61]:

from ensurepip import version
import math
import itertools

from typing import List, Dict
from collections import defaultdict
from dataclasses import dataclass, field

from Bio.PDB import Superimposer

@dataclass
class Peptide(object):

    accession_id: str
    sequence: str
    graph: Dict[str, list] = field(default_factory=defaultdict(list))
    c_alpha_coordinates: List[List[float]] = field(default_factory=list)
    c_beta_coordinates: List[List[float]] = field(default_factory=list)

    # A recursive function that uses
    # visited[] and parent to detect
    # cycle in subgraph reachable from vertex v.
    def __is_cyclic_util(self, v, edges, visited, parent):
 
        # Mark the current node as visited
        visited[v] = True
 
        # Recur for all the vertices
        # adjacent to this vertex

        for edge in edges:
            for i, node in enumerate(self.graph[edge]):
 
                # If the node is not
                # visited then recurse on it
                if visited[i] == False:
                    tmp_vertices: List = list(self.graph[node])
                    if(self.is_cyclic_util(i, tmp_vertices, visited, v)):
                        return True
                    # If an adjacent vertex is
                    # visited and not parent
                    # of current vertex,
                    # then there is a cycle
                elif parent != i:
                    return True
 
        return False

    def is_cyclic(self) -> bool:

        ### This is not going to work
        ### Find all connected pairs, the calculate the distance between them
        ### Easiest way to is build a graph of connected residues, and only model the 
        ### Ca atoms
        ### The graph is undirected

        # Returns true if the graph
        # contains a cycle, else false.
 
        # Mark all the vertices
        # as not visited

        vertices: List = list(self.graph.values())

        visited = [False]*(len(vertices) + 1)
 
        # Call the recursive helper
        # function to detect cycle in different
        # DFS trees
        for i in range(len(vertices)):
 
            # Don't recur for u if it
            # is already visited
            if visited[i] == False:
                if(self.__is_cyclic_util(i, vertices[i], visited, -1)) == True:
                    return True
 
        return False

    def __qcp(self, coord_two: List) -> float:
        superimporser = Superimposer()
        superimporser.set_atoms(self.c_alpha_coordinates, coord_two)

        return round(superimporser.rms, 3)
    

    def __eq__(self, other):
        ### Use RMSD and sequence to compare two peptides
        ### RMSD of 1 and perfect sequence match
        
        if isinstance(other, Peptide):
            if len(self.c_alpha_coordinates) != len(other.c_alpha_coordinates):
                return False
            rmsd: float = self.__qcp(other.c_alpha_coordinates)

            return True if rmsd >= 0.99 and self.sequence == other.sequence else False



In [62]:
def build_graph(graph: Dict, node_one: str, node_two: str) -> Dict:
    graph[node_one].append(node_two)
    graph[node_two].append(node_one)

    return graph

def calculate_distance(point_one: List[float], point_two: List[float]) -> float:
    dist_x: float = point_one[0] - point_two[0]
    dist_y: float = point_one[1] - point_two[1]
    dist_z: float = point_one[2] - point_two[2]

    ### peptide bond is ~1.32 Angstrom
    ### We are calculating the distance Ca-N-Ca, hence this is ~2.64
    ### We'll use a distance of 2.8 to have a margin of error

    return math.sqrt(dist_x**2 * dist_y**2 * dist_z**2)

In [63]:
### download sequences from the PDB database

import os
import time
import urllib
import itertools
import tempfile

import numpy as np
from Bio.PDB.PDBParser import PDBParser

from typing import List, Set, Dict
from collections import defaultdict

structures: List = []

downloadurl: str = "https://files.rcsb.org/download/{0}.pdb"
parser = PDBParser()


with open(os.path.join(os.getcwd(), "data/pdbs.txt"), "r") as pdbs:
    for pdb in list(pdbs)[:5]:
        url: str
        seq: str
        alpha_atoms: List
        beta_atoms: List

        pdb = pdb.strip()
        if pdb == "":
            continue
        else:
            url: str = downloadurl.format(pdb)

        with tempfile.NamedTemporaryFile(mode='w', delete=False) as pdb_file:
            urllib.request.urlretrieve(url, pdb_file.name)

            structure = parser.get_structure('XXXX', pdb_file.name)
            for model in structure:
                distinct_peptides: Set = set()
                for chain in model:
                    if len(list(chain.get_residues())) < 30:
                        seq = "-".join(s.resname for s in chain.get_residues() if s.resname != "HOH" and s.resname != "NH2")
                        alpha_atoms = [atom for atom in chain.get_atoms() if atom.name == "CA"]
                        beta_atoms = [atom for atom in chain.get_atoms() if atom.name == "CB"]

                        g: Dict[str, List] = defaultdict(list)

                        residues: List = itertools.combinations(list(chain.get_residues()), 2)
                        for residue_one, residue_two in residues:

                            if (residue_one.get_resname() != "NH2" and residue_one.get_resname() != "HOH" and residue_one.get_resname() != "200") and \
                                (residue_two.get_resname()!= "NH2" and residue_two.get_resname() != "HOH" and residue_two.get_resname() != "200"):

                                res_one_name, res_two_name = residue_one.get_resname(), residue_two.get_resname()

                                alpha_one: List[float] = [list(atom.get_coord()) for atom in residue_one.get_atoms() if atom.name == "CA"]
                                alpha_two: List[float] = [list(atom.get_coord()) for atom in residue_two.get_atoms() if atom.name == "CA"]

                                if alpha_one and alpha_two:
                                    if calculate_distance(alpha_one[0], alpha_two[0]) <= 3:
                                        g = build_graph(g, res_one_name, res_two_name)

                        peptide = Peptide(pdb, seq, g, alpha_atoms, beta_atoms)
                        structures.append(peptide)


c:\Python310\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2550.
  warnings.warn(
c:\Python310\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 2742.
  warnings.warn(
c:\Python310\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 18243.
  warnings.warn(
c:\Python310\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 18382.
  warnings.warn(
c:\Python310\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 18547.
  warnings.warn(
c:\Python310\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 18720.
  warnings.warn(
c:\Python310\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarnin

In [68]:
d3to1: Dict[str, str] = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
                            'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
                            'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
                            'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

In [78]:

import random
import peptides
import pandas as pd

random.seed(42)

seqs: List[str] = []

for structure in structures[:10]:

    seq: str = ""
    for s in structure.sequence.split('-'):
        if s in d3to1:
            seq = ''.join([seq, d3to1[s]])
        else:
            non_standard: str = random.choice(["O", "U"])
            seq = ''.join([seq, non_standard])

    pep = peptides.Peptide(seq)

    seqs.append(seq)

    # print(f"Aliphatic index: {pep.aliphatic_index(): .3f}, BOMAN: {pep.boman(): .3f}, Isoelectric Point: {pep.isoelectric_point(): .3f}, Charge @ pH 7.4: {pep.charge(): .3f}")
    print(f"Hydrophobicity: {pep.hydrophobicity(): .3f}, Instability Index: {pep.instability_index(): .3f}, Mass Shift: {pep.mass_shift(): .3f}")

Hydrophobicity:  1.280, Instability Index:  28.260, Mass Shift:  0.000
Hydrophobicity: -0.367, Instability Index:  2.883, Mass Shift:  12.040
Hydrophobicity: -0.367, Instability Index:  2.883, Mass Shift:  12.040
Hydrophobicity: -0.367, Instability Index:  2.883, Mass Shift:  12.040
Hydrophobicity: -0.367, Instability Index:  2.883, Mass Shift:  12.040
Hydrophobicity: -0.367, Instability Index:  2.883, Mass Shift:  12.040
Hydrophobicity: -0.367, Instability Index:  2.883, Mass Shift:  12.040
Hydrophobicity: -0.367, Instability Index:  2.883, Mass Shift:  12.040
Hydrophobicity: -0.367, Instability Index:  2.883, Mass Shift:  12.040
Hydrophobicity: -0.367, Instability Index:  2.883, Mass Shift:  12.040


In [72]:
df = pd.DataFrame([peptides.Peptide(s).descriptors() for s in seqs ])

print(df.head(10))

    BLOSUM1   BLOSUM2   BLOSUM3   BLOSUM4   BLOSUM5   BLOSUM6   BLOSUM7  \
0 -0.332000 -0.554000 -0.226000 -0.196000 -0.362000 -0.304000  0.194000   
1  0.026667 -0.304167  0.755833  0.036667  0.638333 -0.304167 -0.089167   
2  0.026667 -0.304167  0.755833  0.036667  0.638333 -0.304167 -0.089167   
3  0.026667 -0.304167  0.755833  0.036667  0.638333 -0.304167 -0.089167   
4  0.026667 -0.304167  0.755833  0.036667  0.638333 -0.304167 -0.089167   
5  0.026667 -0.304167  0.755833  0.036667  0.638333 -0.304167 -0.089167   
6  0.026667 -0.304167  0.755833  0.036667  0.638333 -0.304167 -0.089167   
7  0.026667 -0.304167  0.755833  0.036667  0.638333 -0.304167 -0.089167   
8  0.026667 -0.304167  0.755833  0.036667  0.638333 -0.304167 -0.089167   
9  0.026667 -0.304167  0.755833  0.036667  0.638333 -0.304167 -0.089167   

   BLOSUM8  BLOSUM9  BLOSUM10  ...     VHSE4   VHSE5  VHSE6     VHSE7  VHSE8  \
0   -0.036  -0.1220     0.058  ... -0.532000  0.0860 -0.822 -0.070000  0.582   
1    0.140   0

In [80]:
print(list(df.columns.values))

['BLOSUM1', 'BLOSUM2', 'BLOSUM3', 'BLOSUM4', 'BLOSUM5', 'BLOSUM6', 'BLOSUM7', 'BLOSUM8', 'BLOSUM9', 'BLOSUM10', 'PP1', 'PP2', 'PP3', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'KF1', 'KF2', 'KF3', 'KF4', 'KF5', 'KF6', 'KF7', 'KF8', 'KF9', 'KF10', 'MSWHIM1', 'MSWHIM2', 'MSWHIM3', 'E1', 'E2', 'E3', 'E4', 'E5', 'ProtFP1', 'ProtFP2', 'ProtFP3', 'ProtFP4', 'ProtFP5', 'ProtFP6', 'ProtFP7', 'ProtFP8', 'SV1', 'SV2', 'SV3', 'SV4', 'ST1', 'ST2', 'ST3', 'ST4', 'ST5', 'ST6', 'ST7', 'ST8', 'T1', 'T2', 'T3', 'T4', 'T5', 'VHSE1', 'VHSE2', 'VHSE3', 'VHSE4', 'VHSE5', 'VHSE6', 'VHSE7', 'VHSE8', 'Z1', 'Z2', 'Z3', 'Z4', 'Z5']
